In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import joblib

# 1. Cargar datos desde el Excel
data_path = "Life_Expectancy_Data.xlsx"
df = pd.read_excel(data_path)

# 2. Limpiar nombres de columnas
df.columns = [c.strip() for c in df.columns]

# 3. Detectar la columna de esperanza de vida (Life Expectancy)
target_candidates = [c for c in df.columns
                     if "life" in c.lower() and "expect" in c.lower()]

if not target_candidates:
    raise ValueError("No se encontró una columna que parezca 'Life expectancy'.")

target_col = target_candidates[0]

# 4. Eliminar filas sin dato de esperanza de vida
df = df.dropna(subset=[target_col])

# 5. Eliminar 'Status' si existe (para simplificar)
if "Status" in df.columns:
    df = df.drop(columns=["Status"])

# 6. Construir X (features) e y_cont (objetivo continua)
X = df.drop(columns=[target_col])
y_continua = df[target_col]

# 7. One-Hot Encoding de Country (si está presente)
if "Country" in X.columns:
    X = pd.get_dummies(X, columns=["Country"], drop_first=True)

# 8. Rellenar NaN en columnas numéricas con la mediana
numeric_cols = X.select_dtypes(include=[np.number]).columns
X[numeric_cols] = X[numeric_cols].fillna(X[numeric_cols].median())

# 9. Crear variable objetivo categórica (rangos de esperanza de vida)
etiquetas_clases = [
    "Esperanza de vida muy baja",
    "Esperanza de vida baja",
    "Esperanza de vida media",
    "Esperanza de vida alta",
    "Esperanza de vida muy alta"
]
y = pd.qcut(y_continua, q=5, labels=etiquetas_clases)

# 10. División en entrenamiento / prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# 11. Escalado de variables
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# 12. Búsqueda sencilla de un buen k
mejor_k = None
mejor_acc = -1.0

for k in range(3, 26, 2):  # 3,5,7,...,25
    modelo_temp = KNeighborsClassifier(n_neighbors=k)
    modelo_temp.fit(X_train_scaled, y_train)
    pred_temp = modelo_temp.predict(X_test_scaled)
    acc = accuracy_score(y_test, pred_temp)

    if acc > mejor_acc:
        mejor_acc = acc
        mejor_k = k

print(f"Mejor k encontrado: {mejor_k}  |  Accuracy: {mejor_acc:.4f}")

# 13. Entrenar modelo final con el mejor k
knn_model = KNeighborsClassifier(n_neighbors=mejor_k)
knn_model.fit(X_train_scaled, y_train)

# 14. Evaluación final
y_pred = knn_model.predict(X_test_scaled)
accuracy_final = accuracy_score(y_test, y_pred)
matriz_conf = confusion_matrix(y_test, y_pred, labels=etiquetas_clases)

print("\nRESULTADOS FINALES KNN")
print("-----------------------")
print(f"Exactitud (accuracy): {accuracy_final:.4f}")
print("Matriz de confusión (filas = real, columnas = predicho):")
print(pd.DataFrame(matriz_conf, index=etiquetas_clases, columns=etiquetas_clases))

# 15. Guardar artefactos para el backend
joblib.dump(knn_model,          "life_knn_model.pkl")
joblib.dump(scaler,             "life_knn_scaler.pkl")
joblib.dump(etiquetas_clases,   "life_knn_classes.pkl")
joblib.dump(X.columns.tolist(), "life_knn_features.pkl")

print("\nArchivos guardados:")
print("- life_knn_model.pkl")
print("- life_knn_scaler.pkl")
print("- life_knn_classes.pkl")
print("- life_knn_features.pkl")



Mejor k encontrado: 3  |  Accuracy: 0.8515

RESULTADOS FINALES KNN
-----------------------
Exactitud (accuracy): 0.8515
Matriz de confusión (filas = real, columnas = predicho):
                            Esperanza de vida muy baja  \
Esperanza de vida muy baja                         108   
Esperanza de vida baja                               7   
Esperanza de vida media                              0   
Esperanza de vida alta                               0   
Esperanza de vida muy alta                           0   

                            Esperanza de vida baja  Esperanza de vida media  \
Esperanza de vida muy baja                      10                        0   
Esperanza de vida baja                         105                        5   
Esperanza de vida media                         11                       90   
Esperanza de vida alta                           1                       18   
Esperanza de vida muy alta                       0                        6   
